# Sisällysluettelo
- [Tietoa ohjeista](../../../../code/01.Introduce)
- [Asennus](../../../../code/01.Introduce)
- [Rajoittamaton generointi](../../../../code/01.Introduce)
- [Puhuminen Phi 3:n puolesta](../../../../code/01.Introduce)
- [Regex](../../../../code/01.Introduce)
- [Valitse](../../../../code/01.Introduce)
- [Ajatusketju](../../../../code/01.Introduce)
- [JSON-generointi](../../../../code/01.Introduce)
- [HTML-generointi](../../../../code/01.Introduce)


# Tietoa Guidance-kirjastosta
Guidance on todistettu avoimen lähdekoodin Python-kirjasto, joka mahdollistaa minkä tahansa kielimallin (LM) tuottamien tulosten hallinnan. Yhdellä API-kutsulla voit ilmaista (Pythonissa) tarkat ohjelmalliset rajoitukset, joita mallin tulee noudattaa, ja luoda rakenteellisen tuloksen JSON-, Python-, HTML-, SQL-muodossa tai missä tahansa rakenteessa, jota käyttötapaus vaatii.

Guidance eroaa perinteisistä kehotetekniikoista. Se asettaa rajoituksia ohjaamalla mallia token kerrallaan inferenssikerroksessa, mikä tuottaa laadukkaampia tuloksia ja vähentää kustannuksia ja viivettä jopa 30–50 %, kun sitä käytetään erittäin rakenteellisissa skenaarioissa.

Lisätietoja Guidance-kirjastosta löydät [julkisesta GitHub-repositorysta](https://github.com/guidance-ai/guidance) tai katso [Guidance Breakout Session](https://www.youtube.com/watch?v=qXMNPVVlCMs) Microsoft Build -tapahtumassa.


# Asennus
1. Asenna Guidance komennolla `pip install guidance --pre`
2. Ota Phi 3.5 mini -päätepiste käyttöön Azuren kautta siirtymällä osoitteeseen https://ai.azure.com/explore/models/Phi-3.5-mini-instruct/version/2/registry/azureml ja napsauttamalla "Deploy"-painiketta
3. Tallenna päätepisteesi API-avain ympäristömuuttujaan nimeltä `AZURE_PHI3_KEY` ja URL-osoite ympäristömuuttujaan nimeltä `AZURE_PHI3_URL`


In [ ]:
from guidance import gen, select, regex, user, assistant, system, json
from guidance.models import AzureGuidance
from json import loads as load_json_str
import os

phi3_url = os.getenv("AZURE_PHI3_URL")
phi3_api_key = os.getenv("AZURE_PHI3_KEY")
phi3_lm = AzureGuidance(f"{phi3_url}/guidance#auth={phi3_api_key}")

# Or, load from HuggingFace to run locally
# from guidance.models import Transformers
# phi3_lm = Transformers("microsoft/Phi-3-mini-4k-instruct")

# Rajoittamaton generointi
Tekstiä voidaan tuottaa ilman rajoituksia käyttämällä `gen()`-funktiota. Tämä vastaa mallin käyttöä ilman Guidancea.

## Keskustelun muotoilu
Kuten monet keskustelumallit, Phi-3 odottaa käyttäjän ja avustajan viestien olevan tietyssä muodossa. Guidance tukee Phi-3:n keskustelumallia ja huolehtii keskustelun muotoilusta puolestasi. Keskusteluvuorojen luomiseksi sijoita jokainen keskustelun osa `with user()`- tai `with assistant()`-lohkoon. `with system()`-lohkoa voidaan käyttää järjestelmäviestin asettamiseen.


In [22]:
lm = phi3_lm
with system():
    lm += "You are a helpful assistant. You have a cranky yet entertaining temperament."
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += gen(temperature=0.8, max_tokens=100)

## Tokenisäästöt
Hyvin jäsennellyissä tilanteissa Guidance voi ohittaa tokenit ja luoda vain tarvittavat tokenit, parantaen suorituskykyä, lisäten tehokkuutta ja säästäen API-kustannuksia. Luodut tokenit näytetään tässä muistikirjassa korostetulla taustalla. Pakotetut tokenit näytetään ilman korostusta ja maksavat saman verran kuin syötetokenit, joiden kustannukset arvioidaan olevan noin kolmasosa ulostulotokenien kustannuksista.

*Huom:* Ensimmäinen esimerkki rajoittamattomalla generoinnilla ei pystynyt pakottamaan mitään tokeneita, koska emme antaneet mitään rajoituksia.


# Puhuminen Phi 3:lle  
Ohjauksen avulla voit helposti lisätä tekstiä mallin vastauksiin. Tämä voi olla hyödyllistä, jos haluat ohjata mallin tuotosta tiettyyn suuntaan.


In [5]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + gen(temperature=0.8, max_tokens=50)

# Rajoittaminen regexillä
Edellisessä esimerkissä Phi 3 vastasi kysymykseen `Canberra` ja antoi sen jälkeen lisäselityksiä. Mallin tuottaman vastauksen rajoittamiseksi tarkalleen yhteen sanaan voidaan käyttää regexiä.


In [6]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + regex("[A-Z][a-z]+")

Regexillä tuotetaan vain sana `Canberra`.


# Valitseminen useista vaihtoehdoista
Kun jotkin mahdolliset vaihtoehdot ovat tiedossa, voit käyttää `select()`-funktiota, jotta malli voi valita vaihtoehtojen listasta.


In [23]:
lm = phi3_lm
with user():
    lm += "What is the capital of Australia?"
with assistant():
    lm += "The capital of Australia is " + select(["Washington", "Canberra", "Sydney", "Melbourne"])

`select()`-funktion avulla vain tokeni `Can` luotiin. Koska `Canberra` on ainoa vaihtoehto, joka voi mahdollisesti täydentää vastauksen, jäljellä olevat tokenit pakotettiin.


# Ajatusketju
Ajatusketju on tekniikka, joka voi parantaa mallin tuottaman sisällön laatua rohkaisemalla sitä käsittelemään ongelmaa vaihe vaiheelta. Tyypillisesti lopullisen vastauksen saavuttamiseksi tarvitaan useita kehotuskierroksia. Ensin ohjeistetaan mallia ajattelemaan askel askeleelta. Sen jälkeen kehotetaan mallia antamaan lopullinen vastaus. Tavallisilla chat-päätelmä-API:illa tämä vaatii kaksi API-kutsua, ja mallin tuottama "ajatusketju" veloitetaan kahdesti – ensin lähtötokeneina, kun malli tuottaa sen, ja sitten uudelleen syötetokeneina toisessa kutsussa. Guidance-työkalulla koko monivaiheinen prosessi käsitellään ja veloitetaan osana yhtä API-kutsua, mikä vähentää kustannuksia ja viivettä.


In [8]:
gsm8k_question = "Mark has a garden with flowers. He planted plants of three different colors in it. Ten of them are yellow, and there are 80% more of those in purple. There are only 25% as many green flowers as there are yellow and purple flowers. How many flowers does Mark have in his garden?"
lm = phi3_lm
with user():
    lm += gsm8k_question
with assistant():
    lm += "Let's think step by step. " + gen(temperature=0.8, max_tokens=500)
    # Prompt for the final answer, which should be a number. Store the output in an "answer" variable.
    lm += "\nTherefore, the final answer is: " + regex(r"\d+", name="answer")

print(f"Final answer: {lm['answer']}")

Final answer: 35


# JSON-luonti
Guidancea voidaan käyttää takaamaan JSON:n luominen, joka noudattaa JSON-skeemaa tai pydantic-mallia, kuten tässä esitettyä käyttäjäprofiilin skeemaa.


In [16]:
user_json_schema = load_json_str("""{
  "$schema": "http://json-schema.org/draft-07/schema#",
  "title": "User Profile",
  "type": "object",
  "properties": {
    "username": {
      "type": "string"
    },
    "age": {
      "type": "integer"
    },
    "email": {
      "type": "string"
    }
  },
  "additionalProperties": false
}
""")

lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=user_json_schema, temperature=1.0)

In [19]:
from pydantic import BaseModel

class UserProfile(BaseModel):
    username: str
    age: int
    email: str


lm = phi3_lm
with user():
    lm += "Generate a JSON object for a user profile. The profile should include a username, age, email, and nothing more."

with assistant():
    lm += json(schema=UserProfile, temperature=1.0)

## HTML-sivujen luominen

Guidancea voidaan käyttää myös koodin luomiseen ja ohjelmointikielen syntaktisten vaatimusten noudattamiseen. Tässä osiossa luomme pienen Guidance-ohjelman hyvin yksinkertaisten HTML-sivujen kirjoittamiseen.

Jaamme verkkosivun pienempiin osiin, joista jokaisella on oma Guidance-funktionsa. Nämä yhdistetään lopullisessa funktiossamme HTML-sivun luomiseksi.
Tämän jälkeen suoritamme tämän funktion Guidance-yhteensopivaa mallia vastaan Azure AI:ssa.

*Huom:* Tämä ei tule olemaan täysin varusteltu HTML-generaattori; tavoitteena on näyttää, kuinka voit luoda rakenteellista outputia omiin tarpeisiisi.

Aloitamme tuomalla tarvittavat osat Guidance-kirjastosta:


In [ ]:
from guidance import guidance
from guidance.library import (
    zero_or_more,
    any_char_but,
    select,
    capture,
    with_temperature,
)
from guidance.models import Model

HTML-verkkosivut ovat erittäin jäsenneltyjä, ja ohjaamme sivun osia käyttämällä Guidancea.  
Kun vaadimme mallilta tekstiä, meidän on varmistettava, ettei se sisällä mitään, mikä voisi olla tunniste - eli meidän on suljettava pois '<' ja '>' merkit.


In [ ]:
@guidance(stateless=True)
def _gen_text(lm: Model):
    return lm + zero_or_more(any_char_but(["<", ">"]))

Voimme sitten käyttää tätä funktiota tekstin luomiseen mielivaltaisen HTML-tagin sisällä:


In [ ]:
@guidance(stateless=True)
def _gen_text_in_tag(lm: Model, tag: str):
    lm += f"<{tag}>"
    lm += _gen_text()
    lm += f"</{tag}>"
    return lm

Nyt luodaan sivun otsikko. Osana tätä meidän täytyy luoda sivun otsikko:


In [ ]:
@guidance(stateless=True)
def _gen_header(lm: Model):
    lm += "<head>\n"
    lm += _gen_text_in_tag("title") + "\n"
    lm += "</head>\n"
    return lm

HTML-sivun runko täytetään otsikoilla ja kappaleilla.  
Voimme määritellä funktion tekemään kumpaakin:


In [ ]:
@guidance(stateless=True)
def _gen_heading(lm: Model):
    lm += select(
        options=[_gen_text_in_tag("h1"), _gen_text_in_tag("h2"), _gen_text_in_tag("h3")]
    )
    lm += "\n"
    return lm

@guidance(stateless=True)
def _gen_para(lm: Model):
    lm += _gen_text_in_tag("p")
    lm += "\n"
    return lm

Nyt määritetään HTML:n rungon sisältö.  
Tämä käyttää `select()`-funktiota `recurse=True`-parametrilla luodakseen useita otsikoita ja kappaleita:


In [ ]:
@guidance(stateless=True)
def _gen_body(lm: Model):
    lm += "<body>\n"
    lm += select(options=[_gen_heading(), _gen_para()], recurse=True)
    lm += "</body>\n"
    return lm

Seuraavaksi siirrymme funktioon, joka luo täydellisen HTML-sivun.  
Lisäämme HTML-aloitustagin, sitten luomme otsikon, sen jälkeen rungon, ja lopuksi liitämme HTML-päätöstagin:


In [ ]:
@guidance(stateless=True)
def _gen_html(lm: Model):
    lm += "<html>\n"
    lm += _gen_header()
    lm += _gen_body()
    lm += "</html>\n"
    return lm

Tarjoamme käyttäjäystävällisen wrapperin, jonka avulla voimme:
- Asettaa generoinnin lämpötilan
- Tallentaa luodun sivun Model-objektista


In [ ]:
@guidance(stateless=True)
def html(
    lm,
    name: str | None = None,
    *,
    temperature: float = 0.0,
):
    return lm + capture(
        with_temperature(_gen_html(), temperature=temperature),
        name=name,
    )

In [ ]:
lm = phi3_lm

lm += "Create a web page about your life story. Split your uplifting tale into multiple paragraphs with headings:\n"
lm += html(name="html_text", temperature=0.7)

Voimme sitten kirjoittaa tuloksen tiedostoon:


In [ ]:
with open('./sample_page.html', 'w') as html_file:
    html_file.write(lm["html_text"])

Ja [katso tulos](../../../../code/01.Introduce/sample_page.html).



---

**Vastuuvapauslauseke**:  
Tämä asiakirja on käännetty käyttämällä tekoälypohjaista käännöspalvelua [Co-op Translator](https://github.com/Azure/co-op-translator). Vaikka pyrimme tarkkuuteen, huomioithan, että automaattiset käännökset voivat sisältää virheitä tai epätarkkuuksia. Alkuperäinen asiakirja sen alkuperäisellä kielellä tulisi pitää ensisijaisena lähteenä. Kriittisen tiedon osalta suositellaan ammattimaista ihmiskäännöstä. Emme ole vastuussa väärinkäsityksistä tai virhetulkinnoista, jotka johtuvat tämän käännöksen käytöstä.
